In [10]:
import pandas as pd

df = pd.read_csv('./datasets/bestportal.csv')

df.head()

,comment,sent
0,"<span class=""comment""> Смешно, есть асинхронны...","<img src=""/i/lpp-nul.png"">"
1,"<span class=""comment""> Классный магазинчик на ...","<img src=""/i/lpp-plus.png"">"
2,"<span class=""comment""> Звичайно дитячий трансп...","<img src=""/i/lpp-plus.png"">"
3,"<span class=""comment""> Замовив через телефон, ...","<img src=""/i/lpp-plus.png"">"
4,"<span class=""comment""> Странно такое читать, п...","<img src=""/i/lpp-nul.png"">"


In [8]:
df['sent'].unique()
df[df['sent'] != '<img src="/i/lpp-nul.png">']

from langdetect import detect
from bs4 import BeautifulSoup

soup = BeautifulSoup(df.iloc[0, 0], 'html.parser')
soup.text

# detect(df.iloc[0, 0])


' Смешно, есть асинхронный двигатель, давно уже есть генераторы, принцип действия на обычной машине какой?  а еще все вкурсе что генератор может выдавать столько что горит проводка, о чем речь, аккумулятор для страта и всё, просто они незнают как сдлеать так что бы мы платили за расходник как тема с бензином, вот и всё )))) '

In [11]:
   
df['comment'] = df['comment'].apply(lambda x: BeautifulSoup(x, 'html.parser').text)
df.head()

,comment,sent
0,"Смешно, есть асинхронный двигатель, давно уже...","<img src=""/i/lpp-nul.png"">"
1,"Классный магазинчик на Оболони, столько разны...","<img src=""/i/lpp-plus.png"">"
2,Звичайно дитячий транспорт в наш час не дуже ...,"<img src=""/i/lpp-plus.png"">"
3,"Замовив через телефон, доставили до Львова. В...","<img src=""/i/lpp-plus.png"">"
4,"Странно такое читать, потому что мы с мужем у...","<img src=""/i/lpp-nul.png"">"


In [16]:
df['comment'] = df['comment'].apply(lambda x: ' '.join(x.split()))

def lang_detect(x):
    try:
        return detect(x)
    except:
        return 'undef'

df['lang'] = df['comment'].apply(lambda x: lang_detect(x))
df.head()

,comment,sent,lang
0,"Смешно, есть асинхронный двигатель, давно уже ...","<img src=""/i/lpp-nul.png"">",ru
1,"Классный магазинчик на Оболони, столько разных...","<img src=""/i/lpp-plus.png"">",ru
2,Звичайно дитячий транспорт в наш час не дуже д...,"<img src=""/i/lpp-plus.png"">",uk
3,"Замовив через телефон, доставили до Львова. Вс...","<img src=""/i/lpp-plus.png"">",uk
4,"Странно такое читать, потому что мы с мужем уж...","<img src=""/i/lpp-nul.png"">",ru


In [17]:
df[df['lang'] == 'undef']

,comment,sent,lang
2541,:-),"<img src=""/i/lpp-plus.png"">",undef
4337,+1000000,"<img src=""/i/lpp-nul.png"">",undef
4713,+1000000,"<img src=""/i/lpp-nul.png"">",undef
6303,ntmk2000@ukr.net,"<img src=""/i/lpp-minus.png"">",undef
9213,0000,"<img src=""/i/lpp-nul.png"">",undef
10091,+1 :),"<img src=""/i/lpp-nul.png"">",undef
11304,100%,"<img src=""/i/lpp-nul.png"">",undef


In [22]:
df = df[df['sent'] != '<img src="/i/lpp-nul.png">']
df = df[df['lang'] != 'uk']

df[df['lang'] == 'undef']

df.groupby('lang').count()

df = df[df['lang'] == 'ru']
df.head()


,comment,sent,lang
1,"Классный магазинчик на Оболони, столько разных...","<img src=""/i/lpp-plus.png"">",ru
5,"в марте мы с мужем не могли выбрать модель, пр...","<img src=""/i/lpp-plus.png"">",ru
6,08-02-2012 10:23Сделал заказ в данном магазине...,"<img src=""/i/lpp-minus.png"">",ru
7,"""Двери Белоруссии"" реально продвинутая клиент ...","<img src=""/i/lpp-plus.png"">",ru
8,"Да, некоторые ошибочно думают, что «ЧЕРНЫЙ ПИА...","<img src=""/i/lpp-plus.png"">",ru


In [23]:
import string

t = {ord(char): ' ' for char in string.punctuation + string.digits}
df['comment'] = df['comment'].apply(lambda x: x.lower().translate(t))
df.head()

,comment,sent,lang
1,классный магазинчик на оболони столько разных...,"<img src=""/i/lpp-plus.png"">",ru
5,в марте мы с мужем не могли выбрать модель пр...,"<img src=""/i/lpp-plus.png"">",ru
6,сделал заказ в данном магазине...,"<img src=""/i/lpp-minus.png"">",ru
7,двери белоруссии реально продвинутая клиент ...,"<img src=""/i/lpp-plus.png"">",ru
8,да некоторые ошибочно думают что «черный пиа...,"<img src=""/i/lpp-plus.png"">",ru


In [25]:
df['target'] = df['sent'].apply(lambda x: 0 if x == '<img src="/i/lpp-minus.png">' else 1)
df.head()



,comment,sent,lang,target
1,классный магазинчик на оболони столько разных...,"<img src=""/i/lpp-plus.png"">",ru,1
5,в марте мы с мужем не могли выбрать модель пр...,"<img src=""/i/lpp-plus.png"">",ru,1
6,сделал заказ в данном магазине...,"<img src=""/i/lpp-minus.png"">",ru,0
7,двери белоруссии реально продвинутая клиент ...,"<img src=""/i/lpp-plus.png"">",ru,1
8,да некоторые ошибочно думают что «черный пиа...,"<img src=""/i/lpp-plus.png"">",ru,1


In [26]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()

data = cv.fit_transform(df['comment'])
data

<8621x53229 sparse matrix of type '<class 'numpy.int64'>'
	with 442827 stored elements in Compressed Sparse Row format>

In [28]:
target = df['target']
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import  GaussianNB
gn = GaussianNB()

dtr, dts, ttr, tts = train_test_split(data, target)

gn.fit(dtr.toarray(), ttr)



GaussianNB(priors=None)

In [30]:
from sklearn.metrics import accuracy_score
accuracy_score(tts, gn.predict(dts.toarray()))

0.79128014842300554